### Reg. no: 20BAI1055
### Name: Rakesh Kumar K S
### CSE1016 Lab assignment - 8
#### - Create an RNN that can write like shakespeare (finished in lab)
#### - Using what you learnt, train a model to reproduce text from a different dataset(say CBSE text books or any other text book)

I've decided to train the RNN to reproduce stories

Source of the stories text file: http://www.textfiles.com/stories/aesop11.txt

In [141]:
import tensorflow as tf

import numpy as np
import os
import time

In [142]:
!gdown --fuzzy https://drive.google.com/file/d/1II2iXWWfI1--N51ZgyY9Px1Onu1PnlzE/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1II2iXWWfI1--N51ZgyY9Px1Onu1PnlzE
To: /content/lab8_stories.txt
100% 229k/229k [00:00<00:00, 104MB/s]


In [143]:
# Read, then decode for py2 compat.
text = open('lab8_stories.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 229320 characters


In [144]:
# Take a look at the first 250 characters in text
print(text[:250])


Aesop's Fables Translated by George Fyler Townsend 



 
The Wolf and the Lamb 
 
WOLF, meeting with a Lamb astray from the fold, resolved not to
lay violent hands on him, but to find some plea to justify to the
Lamb the Wolf's right to ea


In [145]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

78 unique characters


In [146]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [147]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [148]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[53, 54, 55, 56, 57, 58, 59], [76, 77, 78]]>

In [149]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [150]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [151]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [152]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [153]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(229320,), dtype=int64, numpy=array([ 2,  1, 27, ..., 13,  2,  1])>

In [154]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [155]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))




A
e
s
o
p
'
s
 


In [156]:
seq_length = 100

In [157]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\r' b'\n' b'A' b'e' b's' b'o' b'p' b"'" b's' b' ' b'F' b'a' b'b' b'l'
 b'e' b's' b' ' b'T' b'r' b'a' b'n' b's' b'l' b'a' b't' b'e' b'd' b' '
 b'b' b'y' b' ' b'G' b'e' b'o' b'r' b'g' b'e' b' ' b'F' b'y' b'l' b'e'
 b'r' b' ' b'T' b'o' b'w' b'n' b's' b'e' b'n' b'd' b' ' b'\r' b'\n' b'\r'
 b'\n' b'\r' b'\n' b'\r' b'\n' b' ' b'\r' b'\n' b'T' b'h' b'e' b' ' b'W'
 b'o' b'l' b'f' b' ' b'a' b'n' b'd' b' ' b't' b'h' b'e' b' ' b'L' b'a'
 b'm' b'b' b' ' b'\r' b'\n' b' ' b'\r' b'\n' b'W' b'O' b'L' b'F' b',' b' '
 b'm' b'e' b'e' b't'], shape=(101,), dtype=string)


In [158]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"\r\nAesop's Fables Translated by George Fyler Townsend \r\n\r\n\r\n\r\n \r\nThe Wolf and the Lamb \r\n \r\nWOLF, meet"
b'ing with a Lamb astray from the fold, resolved not to\r\nlay violent hands on him, but to find some ple'
b'a to justify to the\r\nLamb the Wolf\'s right to eat him.  He thus addressed him:\r\n"Sirrah, last year yo'
b'u grossly insulted me."  "Indeed," bleated\r\nthe Lamb in a mournful tone of voice, "I was not then bor'
b'n."  Then\r\nsaid the Wolf, "You feed in my pasture."  "No, good sir," replied\r\nthe Lamb, "I have not y'


In [159]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [160]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [161]:
dataset = sequences.map(split_input_target)

In [162]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"\r\nAesop's Fables Translated by George Fyler Townsend \r\n\r\n\r\n\r\n \r\nThe Wolf and the Lamb \r\n \r\nWOLF, mee"
Target: b"\nAesop's Fables Translated by George Fyler Townsend \r\n\r\n\r\n\r\n \r\nThe Wolf and the Lamb \r\n \r\nWOLF, meet"


In [163]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [164]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [165]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [215]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [216]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 79) # (batch_size, sequence_length, vocab_size)


In [217]:
model.summary()

Model: "my_model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     multiple                  20224     
                                                                 
 gru_6 (GRU)                 multiple                  3938304   
                                                                 
 dense_6 (Dense)             multiple                  80975     
                                                                 
Total params: 4,039,503
Trainable params: 4,039,503
Non-trainable params: 0
_________________________________________________________________


In [218]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [219]:
sampled_indices

array([25, 36, 69, 64, 24, 71, 44,  7, 45,  9, 28, 75,  6, 69, 43, 55, 31,
       54, 30, 39, 46, 72, 33, 29,  5, 25, 64,  1,  0, 51, 72, 44, 58, 33,
       40, 77, 47, 63, 68, 20, 36, 78, 18, 16, 44,  1, 11, 75, 48,  6, 64,
       22, 53, 24, 57, 77, 14, 12, 37, 61, 50, 67, 75, 39, 26, 33, 54, 54,
       68, 54, 50, 43, 45, 21, 56, 16, 26, 28, 63, 61, 65, 19, 13, 78, 43,
        8, 72, 56,  4, 74, 17, 32, 20,  5, 26, 45, 75, 50, 27, 33])

In [220]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ause, said, "Pray do not grieve so; but\r\ngo and take a stone, and place it in the hole, and fancy th'

Next Char Predictions:
 b';Jql:sR\'S)Bw&qQcEbDMTtGC";l\n[UNK]YtRfGNyUkp6Jz42R\n,wV&l8a:ey0-KiXowM?GbbpbXQS7d2?Bkim5.zQ(td!v3F6"?SwXAG'


In [221]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [222]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 79)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.3706875, shape=(), dtype=float32)


In [223]:
tf.exp(example_batch_mean_loss).numpy()

79.09799

In [224]:
model.compile(optimizer='adam', loss=loss)

In [225]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [226]:
EPOCHS = 50

In [227]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
35/35 [==============================] - 4s 58ms/step - loss: 3.7256
Epoch 2/50
35/35 [==============================] - 2s 57ms/step - loss: 2.7524
Epoch 3/50
35/35 [==============================] - 2s 57ms/step - loss: 2.4169
Epoch 4/50
35/35 [==============================] - 2s 57ms/step - loss: 2.2533
Epoch 5/50
35/35 [==============================] - 2s 58ms/step - loss: 2.1414
Epoch 6/50
35/35 [==============================] - 2s 58ms/step - loss: 2.0482
Epoch 7/50
35/35 [==============================] - 2s 58ms/step - loss: 1.9611
Epoch 8/50
35/35 [==============================] - 2s 59ms/step - loss: 1.8818
Epoch 9/50
35/35 [==============================] - 2s 59ms/step - loss: 1.8085
Epoch 10/50
35/35 [==============================] - 2s 59ms/step - loss: 1.7400
Epoch 11/50
35/35 [==============================] - 2s 59ms/step - loss: 1.6766
Epoch 12/50
35/35 [==============================] - 2s 59ms/step - loss: 1.6186
Epoch 13/50
35/35 [==================

In [228]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [229]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [230]:
start = time.time()
states = None
next_char = tf.constant(['Story:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Story:  "Shalloo sove abreaditted, yot then might rish had been
assigned by the asce digned at the sound and wondered what
it forebode, when he was excellent before he
had two lame far off on me?'  
 

The Mountain in Labor 
 
A MOUSE who always lived on the land, by an unlore postible moved heale the
more of an interest in the Pantrars, and recuining the
hearty-for he wished to a place there you would, you shall have an amount of
truth, and if the beasts of the forest fled at his approach.  The Lion
alone challengly had between them.  When the
Wolf came up to a Shepherd and fawned upon the preference over all other mythologists.  His
'Mountain delivered of a Mouse."  
 
1t stood un all over the
marketplace.  One day an old hound said to him:  Why do you make
such an exhibition of the ends.  The work of Jusidence, and when they saw the
Ill said to him, "O two bricks of a Heepon:  "My nour leik bought My mached as Kinger, ase; by
the great beautys of emprose of his grief,  

___________

In [231]:
start = time.time()
states = None
next_char = tf.constant(['Story:', 'Story:', 'Story:', 'Story:', 'Story:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'Story:  "Planu,\r\nmust having hands, for the man is in earnest this time; he no longer\r\ntrusts his friends, but became tangled in the meshes and\r\ndrowned.  With his last breaking his perfect more easily a\r\nsestoponor, and was placed in the library they and again consticuto to in actor as you are\r\ndea?\'  Finners, are young, to give him nothing but a too  uf, simple inlice to the\r\nsheepfold."  \r\n \r\nEvil tenstle Debolies.  He seized him and\r\npurposely placed his wings and childs.  The Man rushed upon\r\nthe Cat," there was no one found to do it.  \r\n \r\n\r\nThe Wolf and the Goat \r\n \r\nA WOLF saw some Fles and decorn:  he\r\nplaced the Horseman in chalos or a complaint to Jupiter, said\r\nto a File, and asked of him the favor of a pitcher, who half from the\r\nenoborite of a fable involved in as safe younges, who have escaped\r\nfrom man, only to throw myself in the wite which of them whose heard the\r\nanimals of the field, the birds of the nuteratury.

In [232]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [255]:
states = None
next_char = tf.constant(['Story:'])
result = [next_char]

for n in range(200):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

Story:  "Shallon will place
you, were forgot into the hole.  The Snake wost loxt laiders sent
quickly cried out toward him to be of good courage in the fourteenth
century, when you make such revenged for


We have successfully managed to train a RNN model that can somewhat recreate stories based on what its trained on